In [1]:
import gc as g

# %reset
from IPython import get_ipython
get_ipython().magic('reset -sf') 

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)

import pickle, sys,os,cv2, numpy as np, pandas as pd, scipy, seaborn as sns, json
from random import shuffle

import matplotlib.pyplot as plt
get_ipython().magic('matplotlib inline')
%matplotlib inline

from tqdm import tqdm
from functools import partial
from collections import Counter
from itertools import cycle
from six.moves import cPickle as pickle
from scipy import interp
from skimage.feature import local_binary_pattern,hog
from skimage import data, exposure

print('Libraries Loaded')

Libraries Loaded


In [15]:
def load_data(path,img_heighta,img_widtha,step):
    
    Images = []
    y_M=[]    
    k=0
    
    print('\nPre-processing Files')
    folder = sorted(os.listdir(path))
    print(folder)
    c = 0
    for file in folder: 
        u = 0
        subpath = os.listdir(os.path.join(path,file))      
        print('\nMajor Type : ',file)
        print('Total # of Patches:',len(subpath))
        for file1 in sorted(subpath):         
            subpath1 = os.path.join(path,file,subpath[c])
            img=cv2.imread(os.path.join(subpath1))
            if step==1:
                Images.append(cv2.resize(img, (img_heighta,img_widtha)))
            else:
                Images.append(img)            
                
            y_M.append(k)
            u+=1
            c+=1
            
        k+=1            
        c=0
    k=0
    
    Images_ = np.asarray(Images) 
    Labels_ = np.asarray(y_M)    
    del Images, y_M

    return Images_,Labels_        


In [16]:
def load_p(data_file,q):
    # Reading dictionary to load train and test data
    print('\nTring to load pickle from %s' % data_file)
    with open(data_file, 'rb') as file:
        datasets = pickle.load(file)
        dataset = datasets['dataset']
    print('\nPickle Loaded Successfully!')

    X_train = dataset['X_train']
    Y_train = dataset['Y_train']
    del dataset

    if q==1:
        print('\nX_train shape:', X_train.shape)
        print('Y_train shape:', Y_train.shape)
    
    return X_train,Y_train

In [17]:
def savepickle(x_train,y_train, data_file):
    print('\nTrying to save pickle to %s' % data_file)
    
    X_train = x_train
    Y_train = y_train
    del x_train,y_train
    
    # creating dictionary to store train and test data
    datasets = {
        'dataset' : {'X_train': X_train,'Y_train': Y_train}}

    with open(data_file, 'wb') as file:
        pickle.dump(datasets, file, protocol=pickle.HIGHEST_PROTOCOL)
        del datasets # to free up memory.
    print('\nPickle Saved Successfully!')

In [19]:
print('\nSelect DataSet')
option_D0a = int(input("\n 1-ICIAR2018 \n 2-Dartmouth \n"))
while option_D0a  not in (1,2,3,4,5):
    option_D0a = int(input("\n 1-ICIAR2018 \n 2-Dartmouth \n"))

print('\nSelect Class Type')
option_D0_C_T = int(input("\n 1-Multi \n 2-Binary \n"))
while option_D0_C_T  not in (1,2):
    option_D0_C_T = int(input("\n 1-Multi \n 2-Binary \n"))
    
print('\nSelect Save or Load')
option_D0_S = int(input("\n 1-Save \n 2-Load \n"))
while option_D0_S  not in (1,2):
    option_D0_S = int(input("\n 1-Save \n 2-Load \n"))
    
img_height, img_width = 40,40

if option_D0a==1:
    print("\n\n\n DataSet-ICIAR2018 \n")
    path  = "D://Taimoor_Datasets\Paper 2\WSI\Processed_Class_Wise\RESULTS_DIRECTORY_TRAIN_ICIAR/"
    if option_D0_C_T==1:
        data_file = 'PICKLESNEW/DataSet_ICIAR_AE_Train.pickle'
        option_D0=2
        num_classes=4
        loss_a = 'categorical_crossentropy'
    elif option_D0_C_T==2:
        data_file = 'PICKLESNEW/DataSet_ICIAR_AE_Train_B.pickle'
        option_D0=2
        num_classes=2
        loss_a = 'binary_crossentropy'

elif option_D0a==2:
    print("\n\n\n DataSet-Dartmouth \n")
    path  = "D://Taimoor_Datasets\Paper 2\WSI\Processed_Class_Wise\RESULTS_DIRECTORY_Dart_LUNG/"
    if option_D0_C_T==1:
        data_file = 'PICKLESNEW/DataSet_DART_AE_Train.pickle'   
        option_D0=2
        num_classes=5
        loss_a = 'categorical_crossentropy'

if option_D0_S==1:
    (X_train_, Y_train_) = load_data(path,img_height, img_width,step=0)
#     savepickle(X_train_,Y_train_,data_file)
    
    print('\nSave Complete')
    
elif option_D0_S==2:
    X_train_,Y_train_ = load_p(data_file,1)    
    unique, counts = np.unique(Y_train_, return_counts=True)
    abc= dict(zip(unique, counts))
    print(abc)
    for i in range(len(abc)):
        print('Class %d: %d' % (i, abc[i]))    

    print('\nLoad Complete')
        


Select DataSet

 1-ICIAR2018 
 2-Dartmouth 
1

Select Class Type

 1-Multi 
 2-Binary 
1

Select Save or Load

 1-Save 
 2-Load 
1



 DataSet-ICIAR2018 


Pre-processing Files
['D1', 'D2', 'D3', 'D4']

Major Type :  D1
Total # of Patches: 3770

Major Type :  D2
Total # of Patches: 1655

Major Type :  D3
Total # of Patches: 25590

Major Type :  D4
Total # of Patches: 1200

Save Complete


In [9]:
print('\nSelect DataSet')
option_D0a = int(input("\n 1-ICIAR2018 \n 2-Dartmouth \n"))
while option_D0a  not in (1,2,3,4,5):
    option_D0a = int(input("\n 1-ICIAR2018 \n 2-Dartmouth \n"))

print('\nSelect Class Type')
option_D0_C_T = int(input("\n 1-Multi \n 2-Binary \n"))
while option_D0_C_T  not in (1,2):
    option_D0_C_T = int(input("\n 1-Multi \n 2-Binary \n"))
    
print('\nSelect Save or Load')
option_D0_S = int(input("\n 1-Save \n 2-Load \n"))
while option_D0_S  not in (1,2):
    option_D0_S = int(input("\n 1-Save \n 2-Load \n"))
    
img_height, img_width = 40,40

if option_D0a==1:
    print("\n\n\n DataSet-ICIAR2018 \n")
    path  = "D://Taimoor_Datasets\Paper 2\WSI\Processed_Class_Wise\RESULTS_DIRECTORY_TRAIN_ICIAR/"
    if option_D0_C_T==1:
        data_file = 'PICKLESNEW/DataSet_ICIAR_AE_Train.pickle'
        option_D0=2
        num_classes=4
        loss_a = 'categorical_crossentropy'
    elif option_D0_C_T==2:
        data_file = 'PICKLESNEW/DataSet_ICIAR_AE_Train_B.pickle'
        option_D0=2
        num_classes=2
        loss_a = 'binary_crossentropy'

elif option_D0a==2:
    print("\n\n\n DataSet-Dartmouth \n")
    path  = "D://Taimoor_Datasets\Paper 2\WSI\Processed_Class_Wise\RESULTS_DIRECTORY_Dart_LUNG/"
    if option_D0_C_T==1:
        data_file = 'PICKLESNEW/DataSet_DART_AE_Train.pickle'   
        option_D0=2
        num_classes=5
        loss_a = 'categorical_crossentropy'

if option_D0_S==1:
    (X_train_, Y_train_) = load_data(path,img_height, img_width,step=0)
    savepickle(X_train_,Y_train_,data_file)
    
    print('\nSave Complete')
    
elif option_D0_S==2:
    X_train_,Y_train_ = load_p(data_file,1)    
    unique, counts = np.unique(Y_train_, return_counts=True)
    abc= dict(zip(unique, counts))
    print(abc)
    for i in range(len(abc)):
        print('Class %d: %d' % (i, abc[i]))    

    print('\nLoad Complete')
        


Select DataSet

 1-ICIAR2018 
 2-Dartmouth 
1

Select Class Type

 1-Multi 
 2-Binary 
1

Select Save or Load

 1-Save 
 2-Load 
2



 DataSet-ICIAR2018 


Tring to load pickle from PICKLESNEW/DataSet_ICIAR_AE_Train.pickle

Pickle Loaded Successfully!

X_train shape: (32215, 40, 40, 3)
Y_train shape: (32215,)
{0: 3770, 1: 1655, 2: 25590, 3: 1200}
Class 0: 3770
Class 1: 1655
Class 2: 25590
Class 3: 1200

Load Complete


In [10]:
print('\nSelect DataSet')
option_D0a = int(input("\n 1-ICIAR2018 \n 2-Dartmouth \n"))
while option_D0a  not in (1,2,3,4,5):
    option_D0a = int(input("\n 1-ICIAR2018 \n 2-Dartmouth \n"))

print('\nSelect Class Type')
option_D0_C_T = int(input("\n 1-Multi \n 2-Binary \n"))
while option_D0_C_T  not in (1,2):
    option_D0_C_T = int(input("\n 1-Multi \n 2-Binary \n"))
    
print('\nSelect Save or Load')
option_D0_S = int(input("\n 1-Save \n 2-Load \n"))
while option_D0_S  not in (1,2):
    option_D0_S = int(input("\n 1-Save \n 2-Load \n"))
    
img_height, img_width = 40,40

if option_D0a==1:
    print("\n\n\n DataSet-ICIAR2018 \n")
    path  = "D://Taimoor_Datasets\Paper 2\WSI\Processed_Class_Wise\RESULTS_DIRECTORY_TRAIN_ICIAR/"
    if option_D0_C_T==1:
        data_file = 'PICKLESNEW/DataSet_ICIAR_AE_Train.pickle'
        option_D0=2
        num_classes=4
        loss_a = 'categorical_crossentropy'
    elif option_D0_C_T==2:
        data_file = 'PICKLESNEW/DataSet_ICIAR_AE_Train_B.pickle'
        option_D0=2
        num_classes=2
        loss_a = 'binary_crossentropy'

elif option_D0a==2:
    print("\n\n\n DataSet-Dartmouth \n")
    path  = "D://Taimoor_Datasets\Paper 2\WSI\Processed_Class_Wise\RESULTS_DIRECTORY_Dart_LUNG/"
    if option_D0_C_T==1:
        data_file = 'PICKLESNEW/DataSet_DART_AE_Train.pickle'   
        option_D0=2
        num_classes=5
        loss_a = 'categorical_crossentropy'

if option_D0_S==1:
    (X_train_, Y_train_) = load_data(path,img_height, img_width,step=0)
    savepickle(X_train_,Y_train_,data_file)
    
    print('\nSave Complete')
    
elif option_D0_S==2:
    X_train_,Y_train_ = load_p(data_file,1)    
    unique, counts = np.unique(Y_train_, return_counts=True)
    abc= dict(zip(unique, counts))
    print(abc)
    for i in range(len(abc)):
        print('Class %d: %d' % (i, abc[i]))    

    print('\nLoad Complete')
        


Select DataSet

 1-ICIAR2018 
 2-Dartmouth 
1

Select Class Type

 1-Multi 
 2-Binary 
2

Select Save or Load

 1-Save 
 2-Load 
2



 DataSet-ICIAR2018 


Tring to load pickle from PICKLESNEW/DataSet_ICIAR_AE_Train_B.pickle

Pickle Loaded Successfully!

X_train shape: (32215, 224, 224, 3)
Y_train shape: (32215,)
{0: 4970, 1: 27245}
Class 0: 4970
Class 1: 27245

Load Complete


In [11]:
print('\nSelect DataSet')
option_D0a = int(input("\n 1-ICIAR2018 \n 2-Dartmouth \n"))
while option_D0a  not in (1,2,3,4,5):
    option_D0a = int(input("\n 1-ICIAR2018 \n 2-Dartmouth \n"))

print('\nSelect Class Type')
option_D0_C_T = int(input("\n 1-Multi \n 2-Binary \n"))
while option_D0_C_T  not in (1,2):
    option_D0_C_T = int(input("\n 1-Multi \n 2-Binary \n"))
    
print('\nSelect Save or Load')
option_D0_S = int(input("\n 1-Save \n 2-Load \n"))
while option_D0_S  not in (1,2):
    option_D0_S = int(input("\n 1-Save \n 2-Load \n"))
    
img_height, img_width = 40,40

if option_D0a==1:
    print("\n\n\n DataSet-ICIAR2018 \n")
    path  = "D://Taimoor_Datasets\Paper 2\WSI\Processed_Class_Wise\RESULTS_DIRECTORY_TRAIN_ICIAR/"
    if option_D0_C_T==1:
        data_file = 'PICKLESNEW/DataSet_ICIAR_AE_Train.pickle'
        option_D0=2
        num_classes=4
        loss_a = 'categorical_crossentropy'
    elif option_D0_C_T==2:
        data_file = 'PICKLESNEW/DataSet_ICIAR_AE_Train_B.pickle'
        option_D0=2
        num_classes=2
        loss_a = 'binary_crossentropy'

elif option_D0a==2:
    print("\n\n\n DataSet-Dartmouth \n")
    path  = "D://Taimoor_Datasets\Paper 2\WSI\Processed_Class_Wise\RESULTS_DIRECTORY_Dart_LUNG/"
    if option_D0_C_T==1:
        data_file = 'PICKLESNEW/DataSet_DART_AE_Train.pickle'   
        option_D0=2
        num_classes=5
        loss_a = 'categorical_crossentropy'

if option_D0_S==1:
    (X_train_, Y_train_) = load_data(path,img_height, img_width,step=0)
    savepickle(X_train_,Y_train_,data_file)
    
    print('\nSave Complete')
    
elif option_D0_S==2:
    X_train_,Y_train_ = load_p(data_file,1)    
    unique, counts = np.unique(Y_train_, return_counts=True)
    abc= dict(zip(unique, counts))
    print(abc)
    for i in range(len(abc)):
        print('Class %d: %d' % (i, abc[i]))    

    print('\nLoad Complete')
        


Select DataSet

 1-ICIAR2018 
 2-Dartmouth 
2

Select Class Type

 1-Multi 
 2-Binary 
1

Select Save or Load

 1-Save 
 2-Load 
2



 DataSet-Dartmouth 


Tring to load pickle from PICKLESNEW/DataSet_DART_AE_Train.pickle

Pickle Loaded Successfully!

X_train shape: (188470, 40, 40, 3)
Y_train shape: (188470,)
{0: 38611, 1: 39092, 2: 40349, 3: 32228, 4: 38190}
Class 0: 38611
Class 1: 39092
Class 2: 40349
Class 3: 32228
Class 4: 38190

Load Complete
